<a href="https://colab.research.google.com/github/divishagupta99/WiFi_Localisation/blob/master/WiFi_Localisation___Quantile_Distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Quantile Distribution Approach for RSSI WiFi Localisation
Preprocess data.
Define a fixed "Locations" array.


In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.spatial.distance import directed_hausdorff
import statistics
from sklearn.preprocessing import QuantileTransformer

In [ ]:

!git clone https://github.com/divishagupta99/WiFi_Localisation.git

import os
def rename(): 
  i = 0  
  for filename in os.listdir("/content/WiFi_Localisation/Data Set/"): 
    dst =str(filename).replace('-','_')
    src ='/content/WiFi_Localisation/Data Set/'+ filename 
    dst ='/content/WiFi_Localisation/Data Set/'+ dst 
      
    # rename() function will 
    # rename all the files 
    os.rename(src, dst) 
    i += 1

rename()

dataframes = []
names = []
def importing_df():
  for filename in os.listdir("/content/WiFi_Localisation/Data Set/"):
    dataframes.append(pd.read_csv("/content/WiFi_Localisation/Data Set/"+str(filename),header=1))
    new_name = str(filename).replace('.csv','')
    names.append(new_name)
def preprocessing():
  for i in range(len(dataframes)):
    #df_temp = dataframes[i]
    dataframes[i].drop(dataframes[i].loc[dataframes[i]['SSID']!= 'BPGC-NAB'].index, inplace=True)      #Considering only BPGC-NAB
    dataframes[i].reset_index(drop = True, inplace=True)
    for j in range(len(dataframes[i]['Strength'])):
      dataframes[i]['Strength'][j] = str(dataframes[i]['Strength'][j]).replace('dBm', '')
    dataframes[i]['Strength'] = dataframes[i]['Strength'].astype(int)
    dataframes[i] = dataframes[i].drop(columns = ['Primary Frequency', 'Distance', '802.11mc', 'Security', 'Time Stamp', 'SSID'])



importing_df()
preprocessing()

BSSID = []
def unique_BSSID():
  for i in range(len(dataframes)):
    df_temp = dataframes[i]
    for j in df_temp['BSSID']:
      BSSID.append(j)

unique_BSSID()
BSSID = list(set(BSSID))
#sorting BBSID
BSSID.sort()
#The orderof BSSID will remain constant 

fatal: destination path 'WiFi_Localisation' already exists and is not an empty directory.


In [ ]:
locations = ['WG','D151', 'D153', 'D154', 'EEEG', 'StairG',
             'D156', 'D160', 'D164', 'D168', 'D170', 'EntranceG',
             'WF', 'EntranceF', 'D251', 'D250', 'D249', 'StairF',
             'D267', 'D269', 'D266', 'D262', 'D258', 'D252']

In [ ]:
def match_adjacent(original_position, predicted_position):
  if (original_position=='WG' and (predicted_position == 'EntranceG' or predicted_position == 'D151')):
    return True
  elif (original_position=='D151' and (predicted_position == 'WG' or predicted_position == 'D153')):
    return True
  elif (original_position=='D153' and (predicted_position == 'D151' or predicted_position == 'D154')):
    return True
  elif (original_position=='D154' and (predicted_position == 'D153' or predicted_position == 'EEEG')):
    return True
  elif (original_position=='EEEG' and (predicted_position == 'D154' or predicted_position == 'StairG')):
    return True
  elif (original_position=='StairG' and (predicted_position == 'EEEG' or predicted_position == 'D156')):
    return True
  elif (original_position=='D156' and (predicted_position == 'StairG' or predicted_position == 'D160')):
    return True
  elif (original_position=='D160' and (predicted_position == 'D156' or predicted_position == 'D164')):
    return True
  elif (original_position=='D164' and (predicted_position == 'D160' or predicted_position == 'D168')):
    return True
  elif (original_position=='D168' and (predicted_position == 'D164' or predicted_position == 'D170')):
    return True
  elif (original_position=='D170' and (predicted_position == 'D168' or predicted_position == 'EntranceG')):
    return True
  elif (original_position=='EntranceG' and (predicted_position == 'D170' or predicted_position == 'WG')):
    return True

  elif (original_position=='EntranceF' and (predicted_position == 'WF' or predicted_position == 'D252')):
    return True
  elif (original_position=='WF' and (predicted_position == 'EntranceF' or predicted_position == 'D251')):
    return True
  elif (original_position=='D251' and (predicted_position == 'WF' or predicted_position == 'D250')):
    return True
  elif (original_position=='D250' and (predicted_position == 'D251' or predicted_position == 'D249')):
    return True
  elif (original_position=='D249' and (predicted_position == 'D250' or predicted_position == 'StairF')):
    return True
  elif (original_position=='StairF' and (predicted_position == 'D249' or predicted_position == 'D267')):
    return True
  elif (original_position=='D267' and (predicted_position == 'StairF' or predicted_position == 'D269' or predicted_position == 'D266')):
    return True
  elif (original_position=='D269' and (predicted_position == 'D267' or predicted_position == 'D266')):
    return True
  elif (original_position=='D266' and (predicted_position == 'D267' or predicted_position == 'D269' or predicted_position == 'D262')):
    return True
  elif (original_position=='D262' and (predicted_position == 'D266' or predicted_position == 'D258')):
    return True
  elif (original_position=='D258' and (predicted_position == 'D262' or predicted_position == 'D252')):
    return True
  elif (original_position=='D252' and (predicted_position == 'D258' or predicted_position == 'EntranceF')):
    return True
    
  else:
    return False


### Quantile Fingerprint Generation: List of 2D Fingerprints, each for a particular location. Each fingerprint has rows corresponding to an AP and columns as multiple values.

### For 30 Quantiles - Without Normalisation

In [ ]:
dataframes_30 = dataframes.copy()
Q = []  #3D array - 2D Fingerprint Quantile for each 24 locations. Rows sorted by BSSID. Columns no need to sort.
def ThreeD(a, b, c): 
    lst = [[ ['#' for col in range(a)] for col in range(b)] for row in range(c)] 
    return lst  
col1 = 30
col2 = 32
row = 24 
Q_train = (ThreeD(col1, col2, row*2)) #48 rows in train
Q_test = (ThreeD(col1, col2, row)) 

print(len(Q_train[0][0]))

30


In [ ]:
#Filling y_train: As quantiles are arranged location wise, y_train are also sorted location-wise 2 times : once for _1 and another for _4
y_train = [0 for col in range(48)]
for i in range(24):
  y_train[i] = locations[i]
  y_train[i+24]=locations[i]

print(y_train)

['WG', 'D151', 'D153', 'D154', 'EEEG', 'StairG', 'D156', 'D160', 'D164', 'D168', 'D170', 'EntranceG', 'WF', 'EntranceF', 'D251', 'D250', 'D249', 'StairF', 'D267', 'D269', 'D266', 'D262', 'D258', 'D252', 'WG', 'D151', 'D153', 'D154', 'EEEG', 'StairG', 'D156', 'D160', 'D164', 'D168', 'D170', 'EntranceG', 'WF', 'EntranceF', 'D251', 'D250', 'D249', 'StairF', 'D267', 'D269', 'D266', 'D262', 'D258', 'D252']


In [ ]:

# Datasets _1, _4 are for training. _2 will be used for testing
#STEP 1: TRAINING
for i in range(len(dataframes_30)):
  if not(names[i][-2:]=='_4' ):
    continue  #because _4 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_30[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_30[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_30[i])):
    bssid = dataframes_30[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name)][bssid_index])):
      if (Q_train[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name)][bssid_index][k]= dataframes_30[i].loc[j,'Strength']
        break

for i in range(len(dataframes_30)):
  if not(names[i][-2:]=='_1'):
    continue  #because _1 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_30[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_30[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_30[i])):
    bssid = dataframes_30[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name) + 24][bssid_index])):
      if (Q_train[locations.index(loc_name) + 24][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name) + 24][bssid_index][k]= dataframes_30[i].loc[j,'Strength']
        break

for i in range(row*2):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -95
    for k in range(col1):
      if (Q_train[i][j][k]=='#'):  #empty cell - add RSSI as -95
        Q_train[i][j][k]= -95

#STEP 2: GENERATING TEST DATASET

y_test = [0 for col in range(24)]
for i in range(24):
  y_test[i] = locations[i]


for i in range(len(dataframes_30)):
  if not(names[i][-2:]=='_2'):
    continue  #because _2 is for testing and this loop is only for testing
  loc_name = names[i][:-2]  #take the position name
  dataframes_30[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_30[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_30[i])):
    bssid = dataframes_30[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_test[locations.index(loc_name)][bssid_index])):
      if (Q_test[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_test[locations.index(loc_name)][bssid_index][k]= dataframes_30[i].loc[j,'Strength']
        break

for i in range(row):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -95
    for k in range(col1):
      if (Q_test[i][j][k]=='#'):  #empty cell - add RSSI as -95
        Q_test[i][j][k]= -95
  


### Quantile Fingerprint generation: Each RSSI value is converted into a quantile value. Then, use PH-distance to compare fingerprints and choose the least one.

In [ ]:
numberOfQ = len(Q_train)
quantile_transformer = QuantileTransformer(n_quantiles=30, random_state=0)

Q_train_trans = Q_train.copy()
Q_test_trans = Q_test.copy()


for i in range(len(Q_train)):
  Q_train_trans[i] = quantile_transformer.fit_transform(np.transpose(Q_train[i]))
  np.transpose(Q_train[i])

for i in range(len(Q_test)):
  Q_test_trans[i] = quantile_transformer.transform(np.transpose(Q_test[i]))
  np.transpose(Q_test[i])

In [ ]:
def Loc_estimation():
  estimatedPositionArray = []
  for i in range(len(Q_test_trans)):  #for each quantile in testing set
    Q1 = Q_test_trans[i]
    estimatedPosition = 0
    reference = np.inf
    for j in range(len(Q_train_trans)):
      Q2 = Q_train_trans[j]
      d = directed_hausdorff(Q1, Q2)[0] #calculates and returns PH distance
      if d<reference:
        reference = d
        estimatedPosition = y_train[j]
    estimatedPositionArray.append(estimatedPosition)
  return estimatedPositionArray


In [ ]:
predictedPositions = Loc_estimation()

In [ ]:
def accuracy_score_PH(predicted):
  sum_ans = 0
  for i in range(len(predicted)):
    if predicted[i] == y_test[i]:     #Original == Predicted exactly
      sum_ans = sum_ans + 2
    elif match_adjacent(predicted[i],y_test[i]): #adjacent positions with lesser weights
      sum_ans = sum_ans + 1
    else:
      continue
  return sum_ans/(2*len(predicted))

def Raccuracy_score_PH(predicted):
  sum_ans = 0
  for i in range(len(predicted)):
    if predicted[i] == y_test[i]:     #Original == Predicted exactly
      sum_ans = sum_ans + 2
    else:
      continue
  return sum_ans/(2*len(predicted))

In [ ]:
for i in range(len(y_test)):
  print(y_test[i] + " , " + predictedPositions[i])
  

WG , WF
D151 , EntranceG
D153 , D153
D154 , StairG
EEEG , StairG
StairG , StairG
D156 , D160
D160 , D160
D164 , D160
D168 , D164
D170 , D170
EntranceG , EntranceG
WF , D251
EntranceF , EntranceG
D251 , D251
D250 , D250
D249 , StairG
StairF , EEEG
D267 , D267
D269 , D269
D266 , D267
D262 , D258
D258 , D258
D252 , D258


In [ ]:
## Without normalisation
print(accuracy_score_PH(predictedPositions))
print(Raccuracy_score_PH(predictedPositions))

0.5833333333333334
0.4166666666666667


###Quantile Transformation with Normalised Values and number of quantiles = 30

In [ ]:
dataframes_30 = dataframes.copy()

##Normalization: 
rssi_1=[]
rssi_2=[]
rssi_4=[]

def normalise_dataframe_30():
  for i in range(len(dataframes_30)):
    if names[i][-2:]=='_3':
      continue  #Not considering this device due to avoid Android Throttling consequences
    for j in range(len(dataframes_30[i])):
      if names[i][-2:]=='_1':
        rssi_1.append(dataframes_30[i].loc[j,'Strength'])
      if names[i][-2:]=='_2':
        rssi_2.append(dataframes_30[i].loc[j,'Strength'])
      if names[i][-2:]=='_4':
        rssi_4.append(dataframes_30[i].loc[j,'Strength'])
  mean_1=statistics.mean(rssi_1)
  mean_2=statistics.mean(rssi_2)
  mean_4=statistics.mean(rssi_4)
  stddev_1 = statistics.stdev (rssi_1)
  stddev_2 = statistics.stdev(rssi_2)
  stddev_4 = statistics.stdev(rssi_4)
  print(mean_1, mean_2, mean_4, stddev_1, stddev_2, stddev_4)
  for i in range(len(dataframes_30)):
    for j in range(len(dataframes_30[i])):
      #print("Reached" + str(j))
      if names[i][-2:]=='_1':
        dataframes_30[i].loc[j,'Strength'] = ((dataframes_30[i].loc[j,'Strength'] - mean_1 )/(stddev_1) )
      if names[i][-2:]=='_2':
        dataframes_30[i].loc[j,'Strength'] = ((dataframes_30[i].loc[j,'Strength'] - mean_2 )/(stddev_2) )
      if names[i][-2:]=='_4':
        dataframes_30[i].loc[j,'Strength'] = ((dataframes_30[i].loc[j,'Strength'] - mean_4 )/(stddev_4) )

normalise_dataframe_30()

Q = []  #3D array - 2D Fingerprint Quantile for each 24 locations. Rows sorted by BSSID. Columns no need to sort.
def ThreeD(a, b, c): 
    lst = [[ ['#' for col in range(a)] for col in range(b)] for row in range(c)] 
    return lst  
col1 = 30
col2 = 32
row = 24 
Q_train = (ThreeD(col1, col2, row*2)) #48 rows in train
Q_test = (ThreeD(col1, col2, row)) 
#Filling y_train: As quantiles are arranged location wise, y_train are also sorted location-wise 2 times : once for _1 and another for _4
y_train = [0 for col in range(48)]
for i in range(24):
  y_train[i] = locations[i]
  y_train[i+24]=locations[i]

# Datasets _1, _4 are for training. _2 will be used for testing
#STEP 1: TRAINING
for i in range(len(dataframes_30)):
  if not(names[i][-2:]=='_4' ):
    continue  #because _4 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_30[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_30[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_30[i])):
    bssid = dataframes_30[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name)][bssid_index])):
      if (Q_train[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name)][bssid_index][k]= dataframes_30[i].loc[j,'Strength']
        break

for i in range(len(dataframes_30)):
  if not(names[i][-2:]=='_1'):
    continue  #because _1 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_30[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_30[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_30[i])):
    bssid = dataframes_30[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name) + 24][bssid_index])):
      if (Q_train[locations.index(loc_name) + 24][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name) + 24][bssid_index][k]= dataframes_30[i].loc[j,'Strength']
        break

for i in range(row*2):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -95
    for k in range(col1):
      if (Q_train[i][j][k]=='#'):  #empty cell - add RSSI as -95
        Q_train[i][j][k]= -2

#STEP 2: GENERATING TEST DATASET

y_test = [0 for col in range(24)]
for i in range(24):
  y_test[i] = locations[i]


for i in range(len(dataframes_30)):
  if not(names[i][-2:]=='_2'):
    continue  #because _2 is for testing and this loop is only for testing
  loc_name = names[i][:-2]  #take the position name
  dataframes_30[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_30[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_30[i])):
    bssid = dataframes_30[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_test[locations.index(loc_name)][bssid_index])):
      if (Q_test[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_test[locations.index(loc_name)][bssid_index][k]= dataframes_30[i].loc[j,'Strength']
        break

for i in range(row):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -95
    for k in range(col1):
      if (Q_test[i][j][k]=='#'):  #empty cell - add RSSI as -95
        Q_test[i][j][k]= -2
  
numberOfQ = len(Q_train)
quantile_transformer = QuantileTransformer(n_quantiles=30, random_state=0)

Q_train_trans = Q_train.copy()
Q_test_trans = Q_test.copy()


for i in range(len(Q_train)):
  Q_train_trans[i] = quantile_transformer.fit_transform(np.transpose(Q_train[i]))
  np.transpose(Q_train[i])

for i in range(len(Q_test)):
  Q_test_trans[i] = quantile_transformer.transform(np.transpose(Q_test[i]))
  np.transpose(Q_test[i])

-74 -66 -64 9.591663046625438 12.288205727444508 11.789826122551595


In [ ]:
predicted_Positions = Loc_estimation()
for i in range(len(y_test)):
  print(y_test[i] + " , " + predictedPositions[i])
  

WG , WF
D151 , EntranceG
D153 , D153
D154 , StairG
EEEG , StairG
StairG , StairG
D156 , D160
D160 , D160
D164 , D160
D168 , D164
D170 , D170
EntranceG , EntranceG
WF , D251
EntranceF , EntranceG
D251 , D251
D250 , D250
D249 , StairG
StairF , EEEG
D267 , D267
D269 , D269
D266 , D267
D262 , D258
D258 , D258
D252 , D258


In [ ]:
print(accuracy_score_PH(predicted_Positions))
print(Raccuracy_score_PH(predicted_Positions))

0.5833333333333334
0.4166666666666667


###Quantile Distribution and PH Distance using Normalized Values and number of Quantiles = 60

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
#dataframes_60 = dataframes.copy()
rssi_1 = []
rssi_2 =[]
rssi_4 = []

def normalise_dataframe_60():
  for i in range(len(dataframes_60)):
    if names[i][-2:]=='_3':
      continue  #Not considering this device due to avoid Android Throttling consequences
    for j in range(len(dataframes_60[i])):
      if names[i][-2:]=='_1':
        rssi_1.append(dataframes_60[i].loc[j,'Strength'])
      if names[i][-2:]=='_2':
        rssi_2.append(dataframes_60[i].loc[j,'Strength'])
      if names[i][-2:]=='_4':
        rssi_4.append(dataframes_60[i].loc[j,'Strength'])
  mean_1=statistics.mean(rssi_1)
  mean_2=statistics.mean(rssi_2)
  mean_4=statistics.mean(rssi_4)
  stddev_1 = statistics.stdev (rssi_1)
  stddev_2 = statistics.stdev(rssi_2)
  stddev_4 = statistics.stdev(rssi_4)
  print(mean_1, mean_2, mean_4, stddev_1, stddev_2, stddev_4)
  for i in range(len(dataframes_60)):
    for j in range(len(dataframes_60[i])):
      #print("Reached" + str(j))
      if names[i][-2:]=='_1':
        dataframes_60[i].loc[j,'Strength'] = ((dataframes_60[i].loc[j,'Strength'] - mean_1 )/(stddev_1) )
      if names[i][-2:]=='_2':
        dataframes_60[i].loc[j,'Strength'] = ((dataframes_60[i].loc[j,'Strength'] - mean_2 )/(stddev_2) )
      if names[i][-2:]=='_4':
        dataframes_60[i].loc[j,'Strength'] = ((dataframes_60[i].loc[j,'Strength'] - mean_4 )/(stddev_4) )

normalise_dataframe_60()

-1.100348651242953e-17 -6.886348817702079e-18 2.9293299910202487e-18 1.0 1.0 1.0000417847666687


In [ ]:
Q = []  #3D array - 2D Fingerprint Quantile for each 24 locations. Rows sorted by BSSID. Columns no need to sort.
def ThreeD(a, b, c): 
    lst = [[ ['#' for col in range(a)] for col in range(b)] for row in range(c)] 
    return lst  
col1 = 60
col2 = 32
row = 24 
Q_train = (ThreeD(col1, col2, row*2)) #48 rows in train
Q_test = (ThreeD(col1, col2, row)) 

y_train = [0 for col in range(48)]
for i in range(24):
  y_train[i] = locations[i]
  y_train[i+24]=locations[i]



# Datasets _1, _4 are for training. _2 will be used for testing
#STEP 1: TRAINING
for i in range(len(dataframes_60)):
  if not(names[i][-2:]=='_4' ):
    continue  #because _4 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_60[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_60[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes[i])):
    bssid = dataframes_60[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name)][bssid_index])):
      if (Q_train[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name)][bssid_index][k]= dataframes_60[i].loc[j,'Strength']
        break

for i in range(len(dataframes_60)):
  if not(names[i][-2:]=='_1'):
    continue  #because _1 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_60[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_60[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_60[i])):
    bssid = dataframes_60[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name) + 24][bssid_index])):
      if (Q_train[locations.index(loc_name) + 24][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name) + 24][bssid_index][k]= dataframes_60[i].loc[j,'Strength']
        break

for i in range(row*2):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -2
    for k in range(col1):
      if (Q_train[i][j][k]=='#'):  #empty cell - add RSSI as -2
        Q_train[i][j][k]= -2

#STEP 2: GENERATING TEST DATASET

y_test = [0 for col in range(24)]
for i in range(24):
  y_test[i] = locations[i]


for i in range(len(dataframes_60)):
  if not(names[i][-2:]=='_2'):
    continue  #because _2 is for testing and this loop is only for testing
  loc_name = names[i][:-2]  #take the position name
  dataframes_60[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_60[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes[i])):
    bssid = dataframes_60[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_test[locations.index(loc_name)][bssid_index])):
      if (Q_test[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_test[locations.index(loc_name)][bssid_index][k]= dataframes_60[i].loc[j,'Strength']
        break

for i in range(row):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -2
    for k in range(col1):
      if (Q_test[i][j][k]=='#'):  #empty cell - add RSSI as -2
        Q_test[i][j][k]= -2

numberOfQ = len(Q_train)
quantile_transformer = QuantileTransformer(n_quantiles=60, random_state=0)

Q_train_trans = Q_train.copy()
Q_test_trans = Q_test.copy()


for i in range(len(Q_train)):
  Q_train_trans[i] = quantile_transformer.fit_transform(np.transpose(Q_train[i]))
  np.transpose(Q_train[i])

for i in range(len(Q_test)):
  Q_test_trans[i] = quantile_transformer.transform(np.transpose(Q_test[i]))
  np.transpose(Q_test[i])
  
Q_train_trans[0]

array([[0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from scipy.spatial.distance import directed_hausdorff

def Loc_estimation():
  estimatedPositionArray = []
  for i in range(len(Q_test_trans)):  #for each quantile in testing set
    Q1 = Q_test_trans[i]
    estimatedPosition = 0
    reference = np.inf
    for j in range(len(Q_train_trans)):
      Q2 = Q_train_trans[j]
      d = directed_hausdorff(Q1, Q2)[0] #calculates and returns PH distance
      if d<reference:
        reference = d
        estimatedPosition = y_train[j]
    estimatedPositionArray.append(estimatedPosition)
  return estimatedPositionArray


In [ ]:
predictedPositions = Loc_estimation()

for i in range(len(y_test)):
  print(y_test[i] + " , " + predictedPositions[i])
  

WG , WF
D151 , EntranceG
D153 , D153
D154 , StairG
EEEG , StairG
StairG , StairG
D156 , D160
D160 , D160
D164 , D164
D168 , D168
D170 , D170
EntranceG , EntranceG
WF , D251
EntranceF , EntranceG
D251 , D251
D250 , D250
D249 , StairG
StairF , StairF
D267 , D267
D269 , D269
D266 , D267
D262 , D258
D258 , D258
D252 , D258


In [ ]:
## Without normalisation
print(accuracy_score_PH(predictedPositions))
print(Raccuracy_score_PH(predictedPositions))

0.6666666666666666
0.5416666666666666


In [ ]:
## With normalisation
print(accuracy_score_PH(predictedPositions))
print(Raccuracy_score_PH(predictedPositions))

0.6666666666666666
0.5416666666666666


### Quantile Distribution comparing PH Distance, with normalised values and No_of_quantiles = 10

In [ ]:
dataframes_10 = dataframes.copy()

rssi_1 = []
rssi_2 =[]
rssi_4 = []

def normalise_dataframe_10():
  for i in range(len(dataframes_10)):
    if names[i][-2:]=='_3':
      continue  #Not considering this device due to avoid Android Throttling consequences
    for j in range(len(dataframes_10[i])):
      if names[i][-2:]=='_1':
        rssi_1.append(dataframes_10[i].loc[j,'Strength'])
      if names[i][-2:]=='_2':
        rssi_2.append(dataframes_10[i].loc[j,'Strength'])
      if names[i][-2:]=='_4':
        rssi_4.append(dataframes_10[i].loc[j,'Strength'])
  mean_1=statistics.mean(rssi_1)
  mean_2=statistics.mean(rssi_2)
  mean_4=statistics.mean(rssi_4)
  stddev_1 = statistics.stdev (rssi_1)
  stddev_2 = statistics.stdev(rssi_2)
  stddev_4 = statistics.stdev(rssi_4)
  print(mean_1, mean_2, mean_4, stddev_1, stddev_2, stddev_4)
  for i in range(len(dataframes_10)):
    for j in range(len(dataframes_10[i])):
      #print("Reached" + str(j))
      if names[i][-2:]=='_1':
        dataframes_10[i].loc[j,'Strength'] = ((dataframes_10[i].loc[j,'Strength'] - mean_1 )/(stddev_1) )
      if names[i][-2:]=='_2':
        dataframes_10[i].loc[j,'Strength'] = ((dataframes_10[i].loc[j,'Strength'] - mean_2 )/(stddev_2) )
      if names[i][-2:]=='_4':
        dataframes_10[i].loc[j,'Strength'] = ((dataframes_10[i].loc[j,'Strength'] - mean_4 )/(stddev_4) )

normalise_dataframe_10()

Q = []  #3D array - 2D Fingerprint Quantile for each 24 locations. Rows sorted by BSSID. Columns no need to sort.
def ThreeD(a, b, c): 
    lst = [[ ['#' for col in range(a)] for col in range(b)] for row in range(c)] 
    return lst  
col1 = 10
col2 = 32
row = 24 
Q_train = (ThreeD(col1, col2, row*2)) #48 rows in train
Q_test = (ThreeD(col1, col2, row)) 

y_train = [0 for col in range(48)]
for i in range(24):
  y_train[i] = locations[i]
  y_train[i+24]=locations[i]



# Datasets _1, _4 are for training. _2 will be used for testing
#STEP 1: TRAINING
for i in range(len(dataframes_10)):
  if not(names[i][-2:]=='_4' ):
    continue  #because _4 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_10[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_10[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_10[i])):
    bssid = dataframes_10[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name)][bssid_index])):
      if (Q_train[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name)][bssid_index][k]= dataframes_10[i].loc[j,'Strength']
        break

for i in range(len(dataframes_10)):
  if not(names[i][-2:]=='_1'):
    continue  #because _1 is for training
  loc_name = names[i][:-2]  #take the position name
  dataframes_10[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_10[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes_10[i])):
    bssid = dataframes_10[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_train[locations.index(loc_name) + 24][bssid_index])):
      if (Q_train[locations.index(loc_name) + 24][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_train[locations.index(loc_name) + 24][bssid_index][k]= dataframes_10[i].loc[j,'Strength']
        break

for i in range(row*2):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -2
    for k in range(col1):
      if (Q_train[i][j][k]=='#'):  #empty cell - add RSSI as -2
        Q_train[i][j][k]= -2

#STEP 2: GENERATING TEST DATASET

y_test = [0 for col in range(24)]
for i in range(24):
  y_test[i] = locations[i]


for i in range(len(dataframes)):
  if not(names[i][-2:]=='_2'):
    continue  #because _2 is for testing and this loop is only for testing
  loc_name = names[i][:-2]  #take the position name
  dataframes_10[i].sort_values("Strength", ascending=False, inplace=True)
  dataframes_10[i].reset_index(drop=True, inplace=True)
  for j in range(len(dataframes[i])):
    bssid = dataframes_10[i].loc[j,'BSSID']  #take MAC
    bssid_index = BSSID.index(bssid) #find its index (to access in Q matrix)
    #add element j[1] to Q
    for k in range(len(Q_test[locations.index(loc_name)][bssid_index])):
      if (Q_test[locations.index(loc_name)][bssid_index][k]=='#'):  #empty cell - add RSSI
        Q_test[locations.index(loc_name)][bssid_index][k]= dataframes_10[i].loc[j,'Strength']
        break

for i in range(row):
  loc_name = names[i][:-2]  #take the position name
  for j in range(col2):
    #check element for '#' and if so then repeat values to -2
    for k in range(col1):
      if (Q_test[i][j][k]=='#'):  #empty cell - add RSSI as -2
        Q_test[i][j][k]= -2

from sklearn.preprocessing import quantile_transform
numberOfQ = len(Q_train)
quantile_transformer = QuantileTransformer(n_quantiles=10, random_state=0)

Q_train_trans = Q_train.copy()
Q_test_trans = Q_test.copy()


for i in range(len(Q_train)):
  Q_train_trans[i] = quantile_transformer.fit_transform(np.transpose(Q_train[i]))
  np.transpose(Q_train[i])

for i in range(len(Q_test)):
  Q_test_trans[i] = quantile_transformer.transform(np.transpose(Q_test[i]))
  np.transpose(Q_test[i])
  
#Q_train_trans[0]

-0.062046342465939305 -0.0072322509784775585 -0.061634920468764336 1.0012021231924837 1.0001228909000397 1.000936545125591


In [ ]:
predictedPositions = Loc_estimation()

for i in range(len(y_test)):
  print(y_test[i] + " , " + predictedPositions[i])
  

WG , EntranceG
D151 , EntranceG
D153 , D154
D154 , StairG
EEEG , EEEG
StairG , StairG
D156 , D160
D160 , D160
D164 , D160
D168 , D160
D170 , D170
EntranceG , EntranceG
WF , D251
EntranceF , D251
D251 , EntranceF
D250 , D251
D249 , D251
StairF , StairF
D267 , StairF
D269 , D269
D266 , D267
D262 , D250
D258 , D258
D252 , D258


In [ ]:
## Without normalisation
print(accuracy_score_PH(predictedPositions))
print(Raccuracy_score_PH(predictedPositions))

0.5208333333333334
0.3333333333333333


In [ ]:
## With normalisation
print(accuracy_score_PH(predictedPositions))
print(Raccuracy_score_PH(predictedPositions))

0.5208333333333334
0.3333333333333333
